In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup 
import time
import pandas as pd
import requests
import sqlite3
import numpy as np
import warnings
warnings.filterwarnings("ignore", message="In a future version")

In [48]:
def get_symbol_id(market):
        con = sqlite3.connect('share_V2.sqlite')
        cur = con.cursor()
        sql = f"""select Information.Symbol,Information.SymbolId  
                from Information Inner join Market on Market.MarketId = Information.MarketId
                where Market.Mname = "{market}"
                """
        cur.execute(sql)
        records = cur.fetchall()
        con.close()
        symbol_id = {}
        for share in records:
            symbol_id[share[0]] = share[1]
        return symbol_id

In [91]:
#----------------------------------------------FINNOMENA WEB SCRAPE--------------------------------------------------#

In [92]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(r"C:\Users\Admin\Desktop\SOFTDEV2\SOFTWARE-DEVELOPMENT-2\ML\Data\chromedriver.exe")
driver.implicitly_wait(30)

def login():
    try:
        driver.get('https://www.finnomena.com/finno-login/')
        time.sleep(1)
        driver.find_element(By.NAME,'email').send_keys('bungaern2545@gmail.com')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
        driver.find_element(By.NAME,'current_password').send_keys('tiwtiwtiw2545')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
    except Exception as e:
        print(e)
login()

def getStockDetails(name):
    sections = ['stock-financial-report','stock-financial-ratio']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    quarters = []
    data = []
    for section in sections:
            obj = {}
            keys = []
            headerElements = driver.find_element(By.CSS_SELECTOR,
        f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            for topic in headerElements.find_elements(By.CLASS_NAME,'topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                keys.append(key)
            contentElements = driver.find_element(By.CSS_SELECTOR, 
                                f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]}"
                obj[key] = values
                data.append(obj)
    return (quarters,data)

def getFinancialDetails(symbol):
    quarterlyOrAnnual,responseData = getStockDetails(name=symbol)
    data = {}
    for d in responseData:
        data = dict(list(data.items()) + list(d.items()))
    data = {key: data[key] for key in ['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE']}
    data['Period'] = quarterlyOrAnnual
    data['SymbolId'] = str(int(get_symbol_id('SET')[symbol + '.BK']))
    df = pd.DataFrame(data)
    df = df.replace('', '0')
    df = df.iloc[:-1]
    df.iloc[:, :7] = df.iloc[:, :7].applymap(lambda x: x.replace(',', '')).astype(float)
    df = df.replace(0.00, np.nan)
    return df

def allSymbolset():
    conn = sqlite3.connect('share_V2.sqlite')
    cursor = conn.cursor()
    cursor.execute("""SELECT I.Symbol 
                        FROM Information as I
                        INNER JOIN Market as M ON I.MarketId = M.MarketId
                        WHERE Mname = 'SET'
                        ORDER BY Symbol ASC""")
    result = cursor.fetchall()
    values = [item[0][:-3] for item in result]
    return values

def FinnomenaDetail():
    values = allSymbolset()
    nameNoData = ['AURA', 'COMM', 'TRUE']
    details = []
    for name in values:
        response = requests.get('https://www.finnomena.com/stock/' + name)
        if (response.status_code == 404) or (name in nameNoData):
            continue
        else:
            df = getFinancialDetails(symbol=name)
            details.append(df)
    df_new = pd.concat(details)
    return df_new

C:\Users\Admin\AppData\Local\Temp\ipykernel_10076\2382621731.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\Admin\Desktop\SOFTDEV2\SOFTWARE-DEVELOPMENT-2\ML\Data\chromedriver.exe")


In [89]:
#----------------------------------------------ALPHA VANTAGE API--------------------------------------------------#

In [90]:
from alpha_vantage.fundamentaldata import FundamentalData

def alphaVantage_data(symbol):
    apikey = '4YLKM5SFV5RXMQCG'
    fd = FundamentalData(apikey, output_format = 'pandas')
    balance_sheet = fd.get_balance_sheet_quarterly(symbol)
    income_state = fd.get_income_statement_quarterly(symbol)
    balance_sheet = balance_sheet[0].T
    income_state = income_state[0].T
    return (balance_sheet, income_state)

def calInfo(data):
    Equity = (data['totalAssets'] - data['totalLiabilities'])
    NetProfit = (data['grossProfit'] - data['operatingExpenses'])
    ROA_value = (data['netIncome'] / data['totalAssets']) * 100
    ROE_value = (data['netIncome'] / data['totalShareholderEquity']) * 100
    data['Equity'] = Equity
    data['NetProfit'] = NetProfit
    data['ROA'] = ROA_value
    data['ROE'] = ROE_value
    return data

def arrangeAlphaVantageDetail(symbol):
    data_balance, data_income = alphaVantage_data(symbol)
    result = pd.concat([data_balance, data_income])
    result = result.T.reset_index()
    new_result = result.loc[:,~result.columns.duplicated(keep='first')]
    new_result = new_result.drop(new_result.columns[[0,2]], axis=1)
    new_result['netIncome'] = result['netIncome']
    new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    new_result = calInfo(new_result)
    new_result['SymbolId'] = str(int(get_symbol_id("NASDAQ")[symbol]))
    result_after = new_result.drop(new_result.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,
                                          31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]]
                                          , axis=1)
    result_after = result_after.round(3)
    result_after = result_after.rename(columns={"fiscalDateEnding": "Period", "totalAssets": "Asset", "totalLiabilities": "TotalDebt",
                                "totalRevenue": "Revenue"})
    result_after = result_after[['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE', 'Period', 'SymbolId']]
    result_after['Period'] = pd.to_datetime(result_after['Period'])
    result_after = result_after[~(result_after['Period'].isna())]
    result_after = result_after.sort_values(by=['Period'], ascending=True).reset_index(drop=True)
    result_after['Period'] = result_after['Period'].dt.quarter.astype(str) +'Q'+ result_after['Period'].dt.year.astype(str)
    return result_after

def allSymbolnasdaq():
    conn = sqlite3.connect('share_V2.sqlite')
    cursor = conn.cursor()
    cursor.execute("""SELECT I.Symbol 
                        FROM Information as I
                        INNER JOIN Market as M ON I.MarketId = M.MarketId
                        WHERE Mname = 'NASDAQ'
                        ORDER BY Symbol ASC""")
    result = cursor.fetchall()
    values = [item[0] for item in result]
    return values

def AlphaVantageDetail():
    values = allSymbolnasdaq()[:50] #ต่อไป 50-100
    nameNoData = ['ACAC', 'ACACU']
    details_alpha = []
    count = 0
    for name in values:
        if name in nameNoData:
            continue
        elif (count == 2):
            time.sleep(65)
            df = arrangeAlphaVantageDetail(name)
            details_alpha.append(df)
            count = 0
        else:
            df2 = arrangeAlphaVantageDetail(name)
            details_alpha.append(df2)
        count+=1
    df_new = pd.concat(details_alpha)
    return df_new

In [53]:
#-------------------------------------------COMBINE ALL DATA TO DATABASE----------------------------------------------#

In [54]:
# finDetail = FinnomenaDetail()

In [55]:
# finDetail

In [56]:
# alphaDetail = AlphaVantageDetail()

In [57]:
# alphaDetail

In [58]:
# final_data = pd.concat([finDetail, alphaDetail])
# final_data

In [59]:
# import sqlite3
# table_name = 'Financial_quarterly'

# conn = sqlite3.connect('share_V2.sqlite')
# query = f'Create table if not Exists {table_name} (Asset number, TotalDebt number, Equity number, Revenue number, NetProfit number, ROA number, ROE number, Period text, SymbolId number)'
# conn.execute(query)
# final_data.to_sql(table_name,conn,if_exists='replace',index=False)
# conn.commit()
# conn.close()

In [60]:
# def combineFinAlpha():
#     finDetail = FinnomenaDetail()
#     alphaDetail = AlphaVantageDetail()
#     final_data = pd.concat([finDetail, alphaDetail])
#     return final_data

In [61]:
# all_data = combineFinAlpha()
# all_data

In [62]:
#-------------------------------------------UPDATE DATA IN DATABASE----------------------------------------------#

In [81]:
def updateAlphaVantage(symbol):
    conn = sqlite3.connect('share_V2.sqlite')
    df_db = pd.read_sql_query("""SELECT Asset,TotalDebt,Equity,Revenue,NetProfit,ROA,ROE,Period,Symbol,fq.SymbolId FROM Financial_quarterly as fq INNER JOIN Information as i on i.SymbolId = fq.SymbolId ;""", conn)
    if symbol in ['ACAC','ACACU']:
        return 'There is no information for this stock.'
    elif symbol not in df_db['Symbol'].unique():
        data = arrangeAlphaVantageDetail(symbol)
        df_db = df_db.drop(columns=['Symbol'])
        df_new = pd.concat([df_db, data])
        df_new.to_sql('Financial_quarterly', conn, if_exists='append', index=False)
        conn.close()
        return 'Update Success.'
    else:
        df_check = pd.read_sql_query(f"""SELECT Asset,TotalDebt,Equity,Revenue,NetProfit,ROA,ROE,Period,Symbol,fq.SymbolId FROM Financial_quarterly as fq INNER JOIN Information as i on i.SymbolId = fq.SymbolId WHERE Symbol = '{str(symbol)}' ;""", conn)
        data_put = arrangeAlphaVantageDetail(symbol).tail(1)
        df_db = df_db.drop(columns=['Symbol'])
        if data_put['Period'].unique() not in df_check['Period'].unique():
            data_put.to_sql('Financial_quarterly', conn, if_exists='append', index=False)
            conn.close()
            return 'Update Success.'
        else:
            return 'This data is up to date.'

In [97]:
def updateFinnomena(symbol):
    conn = sqlite3.connect('share_V2.sqlite')
    df_db = pd.read_sql_query("""SELECT Asset,TotalDebt,Equity,Revenue,NetProfit,ROA,ROE,Period,Symbol,fq.SymbolId FROM Financial_quarterly as fq INNER JOIN Information as i on i.SymbolId = fq.SymbolId""", conn)
    if symbol in ['AURA', 'COMM', 'TRUE']:
        return 'There is no information for this stock.'
    else:
        df_check = pd.read_sql_query(f"""SELECT Asset,TotalDebt,Equity,Revenue,NetProfit,ROA,ROE,Period,Symbol,fq.SymbolId FROM Financial_quarterly as fq INNER JOIN Information as i on i.SymbolId = fq.SymbolId WHERE Symbol = '{str(symbol)+'.BK'}' """, conn)
        data_put = getFinancialDetails(symbol).tail(1)
        df_db = df_db.drop(columns=['Symbol'])
        if data_put['Period'].unique() not in df_check['Period'].unique():
            data_put.to_sql('Financial_quarterly', conn, if_exists='append', index=False)
            conn.close()
            return 'Update Success'
        else:
            return 'This data is up to date.'

In [99]:
# conn = sqlite3.connect('share_V2.sqlite')
# df_db = pd.read_sql_query("""SELECT Asset, TotalDebt, Equity, Revenue, NetProfit, ROA, ROE, Period, Symbol, fq.SymbolId
#                                 FROM Financial_quarterly as fq 
#                                 INNER JOIN Information as i on i.SymbolId = fq.SymbolId ;""", conn)

# df_db = df_db.drop(columns=['Symbol'])
# df_db

In [100]:
# updateAlphaVantage('META')

In [101]:
# updateFinnomena('KBANK')